In [177]:
import pandas as pd
from gurobipy import *

xls = pd.ExcelFile('Proyecto_1_TransporteMaritimo_Dataset1.xlsx', engine='openpyxl')
nombres= xls.sheet_names
df1 = pd.read_excel(xls, nombres[0])
df2 = pd.read_excel(xls, nombres[1])
df3= pd.read_excel(xls, nombres[2])
df4= pd.read_excel(xls, nombres[3])
df5= pd.read_excel(xls, nombres[4])
df6= pd.read_excel(xls, nombres[5])



In [178]:
n_cargos = len(df3)
n_cargos

15

In [179]:
V= list(df1["ID_Barco"]) ## conjunto barcos
n_barcos = len(V)
n_barcos

4

In [180]:

Kv= {} # conjunto capacidades de barcos
for i in range(0,len(V)):
    Kv[i+1]= list(df1["Capacidad"])[i]
Kv

{1: 16500, 2: 5800, 3: 3570, 4: 3570}

In [181]:
nodos_carga= list(df3["ID_Cargo"])

nodos_descarga= [i+len(nodos_carga) for i in range(1,len(nodos_carga)+1)] ##conjunto nodos de descarga
print("nodos de carga", nodos_carga)
print("nodos de descarga", nodos_descarga)

nodos de carga [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
nodos de descarga [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]


In [182]:
cargas = df3[["ID_Cargo","ID_Puerto_Origen", "Tamano", "Costo_SPOT","LT_Carga","RT_Carga"]].rename(columns={"ID_Puerto_Origen": "ID_Puerto ", "LT_Carga":"LT","RT_Carga":"RT"})
descargas = df3[["ID_Cargo","ID_Puerto_Destino", "Tamano", "Costo_SPOT","LT_Descarga","RT_Descarga"]].rename(columns={"ID_Puerto_Destino": "ID_Puerto ", "LT_Descarga":"LT","RT_Descarga":"RT"})
nodos = pd.concat([cargas,descargas],axis=0)
nodos = nodos.reset_index()[["ID_Cargo","ID_Puerto ", "Tamano", "Costo_SPOT","LT","RT"]]
nodos.index+=1
N = nodos.to_dict("index")

puertos_barco = df1["ID_Puerto "]
t_origen = df1["Tiempo_Inicio"].to_dict()
o = {}
d = {}
for i in range(len(puertos_barco.to_dict())):
  o[i+1] = i+1+n_cargos*2 
  N[i+1+n_cargos*2] = {'Costo_SPOT': 0,
  'ID_Cargo': 0,
  'ID_Puerto ': puertos_barco.to_dict()[i],
  'LT': t_origen[i],
  'RT': 10000,
  'Tamano': 0}
  d[i+1]=i+1+n_cargos*2+len(puertos_barco.to_dict())
  N[i+1+n_cargos*2+len(puertos_barco.to_dict())]={'Costo_SPOT': 0,
  'ID_Cargo': 0,
  'ID_Puerto ': 0,
  'LT': 0,
  'RT': 10000,
  'Tamano': 0}
nodos_index = [i for i in N.keys()]

print(N)
print(nodos_index)

{1: {'ID_Cargo': 1, 'ID_Puerto ': 25, 'Tamano': 2368, 'Costo_SPOT': 354894, 'LT': 383, 'RT': 455}, 2: {'ID_Cargo': 2, 'ID_Puerto ': 9, 'Tamano': 6919, 'Costo_SPOT': 536901, 'LT': 149, 'RT': 221}, 3: {'ID_Cargo': 3, 'ID_Puerto ': 7, 'Tamano': 3395, 'Costo_SPOT': 474427, 'LT': 5, 'RT': 77}, 4: {'ID_Cargo': 4, 'ID_Puerto ': 6, 'Tamano': 5688, 'Costo_SPOT': 458905, 'LT': 244, 'RT': 316}, 5: {'ID_Cargo': 5, 'ID_Puerto ': 38, 'Tamano': 3419, 'Costo_SPOT': 644852, 'LT': 338, 'RT': 410}, 6: {'ID_Cargo': 6, 'ID_Puerto ': 10, 'Tamano': 7277, 'Costo_SPOT': 733327, 'LT': 127, 'RT': 199}, 7: {'ID_Cargo': 7, 'ID_Puerto ': 26, 'Tamano': 2733, 'Costo_SPOT': 276767, 'LT': 621, 'RT': 693}, 8: {'ID_Cargo': 8, 'ID_Puerto ': 14, 'Tamano': 6074, 'Costo_SPOT': 1037825, 'LT': 238, 'RT': 310}, 9: {'ID_Cargo': 9, 'ID_Puerto ': 5, 'Tamano': 2714, 'Costo_SPOT': 326529, 'LT': 39, 'RT': 111}, 10: {'ID_Cargo': 10, 'ID_Puerto ': 9, 'Tamano': 1497, 'Costo_SPOT': 420165, 'LT': 698, 'RT': 770}, 11: {'ID_Cargo': 11, 'ID_

In [183]:
compatibilidad = {}
for i in V:
    compatibilidad[i] = [int(j) for j in list(df2.loc[i-1])[1:] if j>0]
compatibilidad

Nv = {}
for i in V:
  Nv[i] = []
  for j in N.keys():
    if N[j]["ID_Cargo"] in compatibilidad[i] or (j == i+n_cargos*2 or j==i+n_cargos*2+len(V)): 
      Nv[i].append(j)
Nv

{1: [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  10,
  11,
  12,
  13,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  25,
  26,
  27,
  28,
  30,
  31,
  35],
 2: [1,
  3,
  4,
  5,
  7,
  9,
  10,
  11,
  12,
  14,
  15,
  16,
  18,
  19,
  20,
  22,
  24,
  25,
  26,
  27,
  29,
  30,
  32,
  36],
 3: [1, 3, 5, 7, 9, 10, 12, 16, 18, 20, 22, 24, 25, 27, 33, 37],
 4: [1, 5, 7, 9, 10, 12, 14, 16, 20, 22, 24, 25, 27, 29, 34, 38]}

In [184]:
Av={}
for v in V:
  Av[v] = [] 
  for i in Nv[v]:
    for j in Nv[v]:
            Av[v].append([i,j])
Av[1]

[[1, 1],
 [1, 2],
 [1, 3],
 [1, 4],
 [1, 5],
 [1, 6],
 [1, 7],
 [1, 8],
 [1, 10],
 [1, 11],
 [1, 12],
 [1, 13],
 [1, 15],
 [1, 16],
 [1, 17],
 [1, 18],
 [1, 19],
 [1, 20],
 [1, 21],
 [1, 22],
 [1, 23],
 [1, 25],
 [1, 26],
 [1, 27],
 [1, 28],
 [1, 30],
 [1, 31],
 [1, 35],
 [2, 1],
 [2, 2],
 [2, 3],
 [2, 4],
 [2, 5],
 [2, 6],
 [2, 7],
 [2, 8],
 [2, 10],
 [2, 11],
 [2, 12],
 [2, 13],
 [2, 15],
 [2, 16],
 [2, 17],
 [2, 18],
 [2, 19],
 [2, 20],
 [2, 21],
 [2, 22],
 [2, 23],
 [2, 25],
 [2, 26],
 [2, 27],
 [2, 28],
 [2, 30],
 [2, 31],
 [2, 35],
 [3, 1],
 [3, 2],
 [3, 3],
 [3, 4],
 [3, 5],
 [3, 6],
 [3, 7],
 [3, 8],
 [3, 10],
 [3, 11],
 [3, 12],
 [3, 13],
 [3, 15],
 [3, 16],
 [3, 17],
 [3, 18],
 [3, 19],
 [3, 20],
 [3, 21],
 [3, 22],
 [3, 23],
 [3, 25],
 [3, 26],
 [3, 27],
 [3, 28],
 [3, 30],
 [3, 31],
 [3, 35],
 [4, 1],
 [4, 2],
 [4, 3],
 [4, 4],
 [4, 5],
 [4, 6],
 [4, 7],
 [4, 8],
 [4, 10],
 [4, 11],
 [4, 12],
 [4, 13],
 [4, 15],
 [4, 16],
 [4, 17],
 [4, 18],
 [4, 19],
 [4, 20],
 [4, 21],
 [

In [185]:
NP_v = {}
for v in V:
    NP_v[v] = [i for i in nodos_carga if i in Nv[v]]

ND_v = {}
for v in V:
    ND_v[v] = [i for i in nodos_descarga if i in Nv[v]]
print(NP_v)
print(ND_v)



{1: [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15], 2: [1, 3, 4, 5, 7, 9, 10, 11, 12, 14, 15], 3: [1, 3, 5, 7, 9, 10, 12], 4: [1, 5, 7, 9, 10, 12, 14]}
{1: [16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 30], 2: [16, 18, 19, 20, 22, 24, 25, 26, 27, 29, 30], 3: [16, 18, 20, 22, 24, 25, 27], 4: [16, 20, 22, 24, 25, 27, 29]}


In [186]:
LT_i = {i:N[i]["LT"] for i in N.keys()}
print(LT_i)
RT_i = {i:N[i]["RT"] for i in N.keys()}
print(RT_i)

{1: 383, 2: 149, 3: 5, 4: 244, 5: 338, 6: 127, 7: 621, 8: 238, 9: 39, 10: 698, 11: 94, 12: 572, 13: 842, 14: 100, 15: 325, 16: 383, 17: 149, 18: 5, 19: 244, 20: 338, 21: 127, 22: 621, 23: 238, 24: 39, 25: 698, 26: 94, 27: 572, 28: 842, 29: 100, 30: 325, 31: 69, 35: 0, 32: 0, 36: 0, 33: 0, 37: 0, 34: 0, 38: 0}
{1: 455, 2: 221, 3: 77, 4: 316, 5: 410, 6: 199, 7: 693, 8: 310, 9: 111, 10: 770, 11: 166, 12: 644, 13: 914, 14: 172, 15: 397, 16: 777, 17: 588, 18: 489, 19: 624, 20: 871, 21: 657, 22: 982, 23: 967, 24: 501, 25: 1198, 26: 497, 27: 1093, 28: 1388, 29: 800, 30: 746, 31: 10000, 35: 10000, 32: 10000, 36: 10000, 33: 10000, 37: 10000, 34: 10000, 38: 10000}


In [192]:
T_ijv={}
for i in N.keys():
    T_ijv[i] = {} 
    for j in N.keys():
        T_ijv[i][j] = {} 
        for v in V:
            cargo_i= N[i]["ID_Cargo"]
            cargo_j = N[j]["ID_Cargo"]
            if i>n_cargos*2+len(V) or j>n_cargos*2+len(V):
                T_ijv[i][j][v] = 0
            else: 
                puerto_origen = N[i]["ID_Puerto "]
                puerto_destino = N[j]["ID_Puerto "]
                T_ijv[i][j][v] = df4.loc[(df4["Puerto_Origen"]==puerto_origen)&(df4["Puerto_Destino"]==puerto_destino)&(df4["ID_Barco"]==v),["Tiempo_Viaje_hrs"]]["Tiempo_Viaje_hrs"].item()
                if 0<i<=n_cargos:
                    T_ijv[i][j][v] += df5.loc[(df5["ID_Barco"]==v)&(df5["ID_Cargo"]==N[i]["ID_Cargo"]),["Tiempo_Origen_hrs"]]["Tiempo_Origen_hrs"].item()
                if n_cargos*2>=i>n_cargos:
                    T_ijv[i][j][v] += df5.loc[(df5["ID_Barco"]==v)&(df5["ID_Cargo"]==N[i]["ID_Cargo"]),["Tiempo_Destino_hrs"]]["Tiempo_Destino_hrs"].item()

In [193]:
T_ijv[35]

{1: {1: 0, 2: 0, 3: 0, 4: 0},
 2: {1: 0, 2: 0, 3: 0, 4: 0},
 3: {1: 0, 2: 0, 3: 0, 4: 0},
 4: {1: 0, 2: 0, 3: 0, 4: 0},
 5: {1: 0, 2: 0, 3: 0, 4: 0},
 6: {1: 0, 2: 0, 3: 0, 4: 0},
 7: {1: 0, 2: 0, 3: 0, 4: 0},
 8: {1: 0, 2: 0, 3: 0, 4: 0},
 9: {1: 0, 2: 0, 3: 0, 4: 0},
 10: {1: 0, 2: 0, 3: 0, 4: 0},
 11: {1: 0, 2: 0, 3: 0, 4: 0},
 12: {1: 0, 2: 0, 3: 0, 4: 0},
 13: {1: 0, 2: 0, 3: 0, 4: 0},
 14: {1: 0, 2: 0, 3: 0, 4: 0},
 15: {1: 0, 2: 0, 3: 0, 4: 0},
 16: {1: 0, 2: 0, 3: 0, 4: 0},
 17: {1: 0, 2: 0, 3: 0, 4: 0},
 18: {1: 0, 2: 0, 3: 0, 4: 0},
 19: {1: 0, 2: 0, 3: 0, 4: 0},
 20: {1: 0, 2: 0, 3: 0, 4: 0},
 21: {1: 0, 2: 0, 3: 0, 4: 0},
 22: {1: 0, 2: 0, 3: 0, 4: 0},
 23: {1: 0, 2: 0, 3: 0, 4: 0},
 24: {1: 0, 2: 0, 3: 0, 4: 0},
 25: {1: 0, 2: 0, 3: 0, 4: 0},
 26: {1: 0, 2: 0, 3: 0, 4: 0},
 27: {1: 0, 2: 0, 3: 0, 4: 0},
 28: {1: 0, 2: 0, 3: 0, 4: 0},
 29: {1: 0, 2: 0, 3: 0, 4: 0},
 30: {1: 0, 2: 0, 3: 0, 4: 0},
 31: {1: 0, 2: 0, 3: 0, 4: 0},
 35: {1: 0, 2: 0, 3: 0, 4: 0},
 32: {1: 0, 2: 0,

In [194]:
C_ijv={}
for i in N.keys():
    C_ijv[i] = {} 
    for j in N.keys():
        C_ijv[i][j] = {} 
        for v in V:
            if i>n_cargos*2+len(V) or j>n_cargos*2+len(V):
                C_ijv[i][j][v] = 0
            else: 
                puerto_origen = N[i]["ID_Puerto "]
                puerto_destino = N[j]["ID_Puerto "]
                C_ijv[i][j][v] = df4.loc[(df4["Puerto_Origen"]==puerto_origen)&(df4["Puerto_Destino"]==puerto_destino)&(df4["ID_Barco"]==v),["Costo_Viaje_libras"]]["Costo_Viaje_libras"].item()
                if 0<i<=n_cargos:
                    C_ijv[i][j][v] += df5.loc[(df5["ID_Barco"]==v)&(df5["ID_Cargo"]==N[i]["ID_Cargo"]),["Costos_Origen_hrs"]]["Costos_Origen_hrs"].item()
                if n_cargos*2>=i>n_cargos:
                    C_ijv[i][j][v] += df5.loc[(df5["ID_Barco"]==v)&(df5["ID_Cargo"]==N[i]["ID_Cargo"]),["Costos_Destino_hrs"]]["Costos_Destino_hrs"].item()

In [198]:
C_ijv[9][1][1]

41757

In [199]:
Qj={N[i]["ID_Cargo"]:N[i]["Tamano"] for i in N.keys() if N[i]["ID_Cargo"]>0}

Cs = df3["Costo_SPOT"].to_dict()
print("empezando con el modelo")

empezando con el modelo


In [200]:
# Model
m = Model("Transporte Maritimo")

# variables
x = m.addVars(nodos_index,nodos_index,V,vtype=GRB.BINARY, name="Arco")
y = m.addVars(nodos_carga,vtype=GRB.BINARY,name="Cargo_Spot")
t = m.addVars(nodos_index,V,vtype=GRB.INTEGER, name="tiempo")
l = m.addVars(nodos_index,V,vtype=GRB.INTEGER,name="Inventario")



In [201]:
# Objective function
#m.setObjective(sum((sum((CTt[k,t]+4.5)*st[k,t] for k in LOG)+sum((CFt[k,t])*zt[k,t] for k in LOG)+(sum(CBt[m,t]*wt[m,t] for m in BOARD))) for t in MONTH), GRB.MINIMIZE)
m.setObjective(sum(sum(C_ijv[i][j][v]*x[i,j,v] for [i,j] in Av[v]) for v in V)+sum(Cs[i-1]*y[i] for i in nodos_carga), GRB.MINIMIZE)

In [202]:
m.addConstrs(((sum(sum(x[i,j,v] + y[i] for j in Nv[v]) for v in V) == 1) for i in nodos_carga), name = "Const_1")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>}

In [203]:
m.addConstrs(((sum(x[o[v],j,v] for j in Nv[v]) == 1) for v in V), name = "Const_2")



{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [204]:
m.addConstrs(((sum(x[i,j,v] for j in Nv[v])-sum(x[j,i,v] for j in Nv[v])==0) for v in V for i in Nv[v] if (i!=n_cargos*2+v and i!=n_cargos*2+v+n_barcos)), name = "Const_3")

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 21): <gurobi.Constr *Awaiting Model Update*>,
 (1, 22): <gurobi.Constr *Awaiting Mode

In [205]:
m.addConstrs(((sum(x[j,d[v],v] for j in Nv[v]) == 1) for v in V), name = "Const_4")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [206]:
m.addConstrs(((l[i,v]+Qj[j]-l[j,v]<=Kv[v]*(1-x[i,j,v])) for v in V for [i,j] in Av[v] if j in NP_v[v]), name = "Const_5")

{(1, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 6): <gurobi.Constr *Awaiting Mode

In [207]:
m.addConstrs(((l[i,v]-Qj[j]-l[j+n_cargos,v]<=Kv[v]*(1-x[i,j+n_cargos,v]))  for v in V for j in NP_v[v] for i in Nv[v] if ([i,j+n_cargos] in Av[v])), name = "Const_6")
#m.addConstrs(((l[i,v]-Qj[j]-l[j+n_cargos,v]<=Kv[v]*(1-x[i,j+n_cargos,v]))  for v in V for j in NP_v[v] for i in Nv[v] if ([i,j+n_cargos] in Av[v])), name = "Const_6")

{(1, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 21): <gurobi.Constr *Awaitin

In [208]:
m.addConstrs(((l[i,v]<=Kv[v]) for v in V for i in NP_v[v]), name = "Const_7_ub")

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 9): <gurobi.Constr *Awaiting Model Update*>,
 (2, 10): <gurobi.Constr *Awaiting Model Upda

In [209]:
m.addConstrs(((l[i,v]>=0) for v in V for i in NP_v[v]), name = "Const_7_lb")

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 9): <gurobi.Constr *Awaiting Model Update*>,
 (2, 10): <gurobi.Constr *Awaiting Model Upda

In [210]:
m.addConstrs(((t[i,v]+T_ijv[i][j][v]-t[j,v]<=(RT_i[i]+T_ijv[i][j][v])*(1-x[i,j,v])) for v in V for [i,j] in Av[v]), name = "Const_8")

{(1, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 21): <gurobi.Constr *Awaitin

In [211]:
m.addConstrs(((sum(x[i,j,v] for j in Nv[v])-sum(x[n_cargos+i,j,v] for j in Nv[v])==0) for v in V for i in NP_v[v]), name = "Const_9")

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 9): <gurobi.Constr *Awaiting Model Update*>,
 (2, 10): <gurobi.Constr *Awaiting Model Upda

In [212]:

m.addConstrs(((t[i,v]+T_ijv[i][n_cargos+i][v]-t[n_cargos+i,v]<=0) for v in V for i in NP_v[v] ), name = "Const_10")

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 9): <gurobi.Constr *Awaiting Model Update*>,
 (2, 10): <gurobi.Constr *Awaiting Model Upda

In [213]:

m.addConstrs(((t[i,v]<=RT_i[i]) for v in V for i in Nv[v]), name = "Const_11_ub")

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 21): <gurobi.Constr *Awaiting Model Update*>,
 (1, 22): <gurobi.Constr *Awaiting Mode

In [214]:
m.addConstrs(((LT_i[i]-t[i,v]<=-1) for v in V for i in Nv[v]), name = "Const_11_lb")

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 21): <gurobi.Constr *Awaiting Model Update*>,
 (1, 22): <gurobi.Constr *Awaiting Mode

In [215]:
# Solve
m.optimize()

valorobjetivo = m.objval

"""for v in m.getVars():
    print('%s %g' % (v.varName, v.x))"""

print('\n Valor óptimo: %8.4f \n' % valorobjetivo)

m.write("Transporte_final.sol")
m.write("model.lp")

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 3995 rows, 6095 columns and 17131 nonzeros
Model fingerprint: 0xef6b2c49
Variable types: 0 continuous, 6095 integer (5791 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [9e+02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+04]
Found heuristic solution: objective 702673.00000
Presolve removed 2116 rows and 4837 columns
Presolve time: 0.07s
Presolved: 1879 rows, 1258 columns, 8300 nonzeros
Variable types: 0 continuous, 1258 integer (1104 binary)

Root relaxation: objective 1.531902e+05, 241 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 153190.191    0   47 702673.000 153190.191  78.2%     -    0s
H    0     0                    530323.00000 153190.191  71.1%     -    0s
H    0     0                    284035.